In [4]:
from collections import defaultdict
import numpy as np
import sys
from enum import Enum
class ACTION(Enum):
    high=1
    low=0 
    
class Winner:
    def __init__(self):
        self.P_H = 0.55
        self.P_L = 0.45
        self.state = 0
        self.done = 0
        self.R = 0
        self.highcost = -50
        self.lowcost = -10
        self.greedy = 0.1
        self.discount=0.9
        self.init_Q()
        self.init_C()
    def reset_game(self):
        self.state=0
        self.done=0
    def ba_policy(self):
        smp = np.random.uniform(0,1)
        if smp <= 0.5:
            ac = ACTION.high
        else:
            ac = ACTION.low
        return ac
        
    #def ta_policy(self):
    def OffpolicyMCControl(self,numepisode):
        self.init_Q()
        self.init_C()
        for i_episode in range(1, numepisode+1):
            if i_episode % 1000 == 0:
                print("\rEpisode {}/{}.".format(i_episode, numepisode), end="")
                sys.stdout.flush()
            
            self.reset_game()
            state=0
            episode = []
            while(1):
                action = self.ba_policy()
                reward,next_state,done = self.env_step(state,action)
                episode.append((state, action, reward))
                if done:
                    break
                state = next_state
            G = 0.0
            Weight = 1.0
            for t in reversed(range(len(episode))):
                state, action, reward = episode[t]
                G = self.discount * G + reward
                self.C[state,action] += Weight
                self.Q[state,action] += (Weight / self.C[state,action]) * (G - self.Q[state,action])
                if action==ACTION.high:
                    if self.Q[state,action]<self.Q[state,ACTION.low]:
                        break
                else:
                    if self.Q[state,action]<self.Q[state,ACTION.high]:
                        break
                Weight = Weight*(2.0)
                        
            
           
        return self.Q
        
        
    def env_step(self,state,action):
        done=0
        reward=0
        sample = np.random.uniform(0, 1)
        if action == ACTION.high:
            reward = reward + self.highcost
            suc_prob = self.P_H
        else:
            suc_prob = self.P_L
            reward = reward + self.lowcost
        if sample < suc_prob:
            state = state + 1
        else:
            state = state -1
        if state >= 3 :
            reward = reward + 1000
            done = 1
        if state <= (-3):
            done = 1
        return reward,state,done
    def init_Q(self):
        self.Q = defaultdict(lambda:np.zeros(1))
        for state in range(7):
            for act in ACTION:
                    self.Q[state-3,act]=0.0
    def init_C(self):
        
        self.C = defaultdict(lambda:np.zeros(1))
        for state in range(7):
            for act in ACTION:
                    self.C[state-3,act]=0.0
        

In [5]:
winner=Winner()

Q=winner.OffpolicyMCControl(500000)
print('policy_value',Q)


Episode 500000/500000.policy_value defaultdict(<function Winner.init_Q.<locals>.<lambda> at 0x7f65e2b88620>, {(-3, <ACTION.high: 1>): 0.0, (-3, <ACTION.low: 0>): 0.0, (-2, <ACTION.high: 1>): -20.126167673179754, (-2, <ACTION.low: 0>): 15.552424495506425, (-1, <ACTION.high: 1>): 40.51452241472253, (-1, <ACTION.low: 0>): 69.32061396079652, (0, <ACTION.high: 1>): 171.69324317406432, (0, <ACTION.low: 0>): 180.2332956082197, (1, <ACTION.high: 1>): 391.0335657868245, (1, <ACTION.low: 0>): 379.80848235669055, (2, <ACTION.high: 1>): 684.1075034326415, (2, <ACTION.low: 0>): 680.2962320029371, (3, <ACTION.high: 1>): 0.0, (3, <ACTION.low: 0>): 0.0})


In [6]:
import pandas as pd
buf = np.zeros((7,2))
for i in range(7):
    for j in ACTION:
        buf[i][j.value]=Q[i-3,j]

test_3 = pd.DataFrame(buf)
print(test_3)

            0           1
0    0.000000    0.000000
1   15.552424  -20.126168
2   69.320614   40.514522
3  180.233296  171.693243
4  379.808482  391.033566
5  680.296232  684.107503
6    0.000000    0.000000
